In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pandas
import sqlalchemy
from dotenv import load_dotenv
import os

# Configuração do Selenium
options = Options()
options.add_argument("--headless")  # Executa em segundo plano
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
print("Ambiente pronto")

Ambiente pronto


In [25]:
load_dotenv()

email = os.getenv("LINKEDIN_EMAIL")
senha = os.getenv("LINKEDIN_PASSWORD")

In [14]:
driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/in/walter-gonzaga/')

In [18]:
driver.find_element(By.CSS_SELECTOR,'#main-content > div > form > p > button').click()

In [ ]:
input_email = driver.find_element(By.ID,'session_key')
input_email.click()
input_email.send_keys(email)

input_senha = driver.find_element(By.ID,'session_password')
input_senha.click()
input_senha.send_keys(senha)

driver.find_element(By.CSS_SELECTOR,'#main-content > div > div.authwall-sign-in-form > form > div.flex.justify-between.sign-in-form__footer--full-width > button').click()


In [27]:
driver.get('https://www.linkedin.com/in/walter-gonzaga/recent-activity/all/')

In [37]:
## Pegar dados iniciais

for i in driver.find_elements(By.CLASS_NAME,'ca-entry-point__num-views'):
    print(i.text)


277 impressões
Visualizar análise
3.102 impressões
Visualizar análise
6.373 impressões
Visualizar análise
1.893 impressões
Visualizar análise
3.580 impressões
Visualizar análise
62.585 impressões
Visualizar análise
1.772 impressões
Visualizar análise


### Coletar URLS de posts

In [ ]:
wait = WebDriverWait(driver, 20)

LOAD_MORE_CSS = "button.scaffold-finite-scroll__load-button"

def tentar_clicar_load_more():
    botoes = driver.find_elements(By.CSS_SELECTOR, LOAD_MORE_CSS)
    for b in botoes:
        if b.is_displayed() and b.is_enabled():
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", b)
            time.sleep(0.4)
            try:
                b.click()
            except:
                driver.execute_script("arguments[0].click();", b)
            return True
    return False

def scroll_ate_carregar(max_rodadas=60):
    ultima_altura = 0

    for _ in range(max_rodadas):
        # desce um pouco (gatilho de lazy render)
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(0.8)

        # se o botão aparecer, clica
        clicou = tentar_clicar_load_more()
        if clicou:
            time.sleep(1.2)

        # checa se ainda está crescendo
        altura = driver.execute_script("return document.body.scrollHeight;")
        if altura == ultima_altura and not clicou:
            break
        ultima_altura = altura

scroll_ate_carregar(max_rodadas=80)


In [ ]:
import re

def coletar_urls_posts():
    links = driver.find_elements(By.CSS_SELECTOR, "a[href]")
    urls = set()

    for a in links:
        href = a.get_attribute("href") or ""
        # pega o padrão mais comum de post no feed
        if "/feed/update/" in href or "urn:li:activity:" in href:
            # limpa parâmetros (opcional)
            href = re.sub(r"\?.*$", "", href)
            urls.add(href)

    return sorted(urls)

urls_posts = coletar_urls_posts()
print("Total URLs:", len(urls_posts))
print(urls_posts[:5])
